In [2]:
import numpy as np
import pandas as pd

PATH =  "/home/jmurga/mkt/201902"
import sys
sys.path.insert(0, PATH + '/scripts/src/')
from analytical import *

In [3]:
PATH = "/home/jmurga/mkt/201902/"

Recipe simulation

In [18]:
!cat $PATH/scripts/slim_recipes/mkt_comparision.slim

initialize() {

	/*defineConstant("theta", 0.02); // 2Nes
	defineConstant("rho", 0.05); // 2Nes
	defineConstant("shape", 0.3); // 2Nes*/
    /*to simulate N=500;alpha 0.4;gH=250;pposH = 0.00024562 Analytical.jl*/
	/*to simulate N=2500;alpha 0.4;gH=250;pposH=0.000191626*/
	/*defineConstant("pposH", 0.000191626);*/
	/*defineConstant("nGenes", 7);*/	
	/*defineConstant("nF", 1);*/
	/*defineConstant("p", '/home/jmurga/test');*/

	defineConstant("exon", 300);
	defineConstant("intron", 100);
	defineConstant("exonNumber", 5);
	
	// m1 mutation type: neutral
	initializeMutationType("m1", 0.5, "f", 0.0);
	// m2 mutation type: deleterious
	initializeMutationType("m2", 0.5, "g", negStrength/(2*N), shape);
	// m3 mutation type: advantageous
	initializeMutationType("m3", 0.5, 'f', posStrength/(2*N));
	// g1 genomic element type: uses m1 for all mutations
	initializeGenomicElementType("g1", c(m1,m2,m3), c(0.25,0.75*(1-(pposH)), 0.75*(pposH)));
	

	// N genes modeled. 5 exons per gene.
	base       = 0

We run the simulation using GNU parallel. Check the file ```../src/run_sim.sh``` to check all the cases. Here we provided an example of the baseline simulation. Each command inside ```../src/run_sim.sh``` will modify one paremeter at a time to test the variable effect on $\alpha$ estimation. We tested the following conditions:
 - rho = [0.001,0.01]
 - theta = [0.001,0.01]
 - Number of genes = [2000,28000]
 - Probability of fixation for selected mutations = [0.000746621,3.5566e-5] ($\alpha = 0.7$ and $\alpha = 0.1$)
 - 2Nes - = [-1000,-500]
 - 2Nes + = [100,500]
 - $\beta$ = [0.1,0.2]

```bash 
parallel -j 20 -u "slim -d N=1000 -d negStrength=-2000 -d posStrength=250 -d shape=0.3 -d rho=0.001 -d theta=0.01 -d pposH=0.000213339 -d nGenes=7 -d nF={} -d p=\'/home/jmurga/test/simulations/impMK/theta_0.01\' /home/jmurga/test/simulations/impMK/mktComparison.slim" ::: `seq 1 2000`
```

We solved the fixation probabilities using a simplified version of [mktest](https://github.com/uricchio/mktest) from [Uricchio et al. 2019](https://www.nature.com/articles/s41559-019-0890-6) whithout BGS nor weak adaptation.

In [11]:
for a in np.array([0.1,0.4,0.7]):
    adap = Analytical(N=1000,gam_neg=-2000,gH=250,al=0.3,be=0.3/2000,alLow=0.0,alTot=a,B=0.999)
    B = adap.B
    adap.set_theta_f()
    theta_f = adap.theta_f
    adap.B = 0.999
    ## Solve the mutation rate
    adap.set_theta_f()
    ## Solve the probabilities
    adap.setPpos()
    print('α = {}, fixation probility = {}'.format(a,adap.pposH))

α = 0.1, fixation probility = 3.556845591841181e-05
α = 0.4, fixation probility = 0.0002133917604722278
α = 0.7, fixation probility = 0.0007466719945885354


In addition we run the baseline simulation accouting for BGS and weak adaptation

In [10]:
adap = Analytical(N=1000,gam_neg=-500,gH=500,gL=5,al=0.3,be=0.3/500,alLow=0.2,alTot=0.4,B=0.999,Lf=1e5)
B = adap.B
adap.set_theta_f()
theta_f = adap.theta_f
adap.B = 0.999
## Solve the mutation rate
adap.set_theta_f()
## Solve the probabilities with no BGS
adap.setPpos()
# Return to the original values
adap.theta_f = theta_f
adap.B = B

print('α = {}, strongy selected alelles = {:.6f}, weakly selected alelles = {:.6f}, B = {}, μ = {}'.format(adap.alTot,adap.pposH,adap.pposL,adap.B,theta_f/(4*adap.N)))

α = 0.4, strongy selected alelles = 0.000093, weakly selected alelles = 0.006904, B = 0.999, μ = 7.503752501876845e-10


In [17]:
adap = Analytical(N=500,gam_neg=-500,gH=500,gL=5,al=0.3,be=0.3/500,alLow=0.2,alTot=0.4,B=0.4,Lf=1e5)
B = adap.B
adap.set_theta_f()
theta_f = adap.theta_f
adap.B = 0.999
## Solve the mutation rate
adap.set_theta_f()
## Solve the probabilities with no BGS
adap.setPpos()
# Return to the original values
adap.theta_f = theta_f
adap.B = B

print('α = {}, strongy selected alelles = {:.6f}, weakly selected alelles = {:.6f}, B = {}, μ = {}'.format(adap.alTot,adap.pposH,adap.pposL,adap.B,theta_f/(4*adap.N)))

α = 0.4, strongy selected alelles = 0.000118, weakly selected alelles = 0.006922, B = 0.4, μ = 1.3744360978112326e-06
